In [1]:
import numpy as np
import pandas as pd

In [2]:
import csv

In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
#makes a dictionary for parcels 
parcels = {}
with open ('Parcels.txt', mode = 'r') as csv_file:
    for line in csv_file:
        line.rstrip('\n')
        (key,val, val2) = line.split(',')
        parcels[int(key)] = [float(val), float(val2)]
parcels
#type(parcels)

{1: [37.519, 337671.0],
 2: [6.62, 19860.0],
 3: [24.126, 48252.0],
 4: [5.379, 80685.0],
 5: [12.252, 49008.0],
 6: [4.839, 14517.0],
 7: [30.92, 185520.0],
 8: [7.555, 30220.0],
 9: [2.043, 8172.0],
 10: [32.031, 0.0],
 11: [1.359, 5436.0],
 12: [12.181, 182715.0],
 13: [25.38, 152280.0],
 14: [37.921, 37921.0],
 15: [7.672, 107408.0],
 16: [34.06, 204360.0],
 17: [12.022, 180330.0],
 18: [10.932, 163980.0],
 19: [5.001, 25005.0],
 20: [30.481, 213367.0],
 21: [9.311, 46555.0],
 22: [3.005, 15025.0],
 23: [8.648, 43240.0],
 24: [3.298, 16490.0],
 25: [20.731, 849971.0],
 26: [2.527, 12635.0],
 27: [5.738, 40166.0],
 28: [3.198, 47970.0],
 29: [6.45, 19350.0],
 30: [7.208, 28832.0],
 31: [26.188, 104752.0],
 32: [14.007, 98049.0],
 33: [11.412, 22824.0],
 34: [18.264, 127848.0],
 35: [10.363, 155445.0],
 36: [9.684, 48420.0],
 37: [18.946, 113676.0],
 38: [13.08, 26160.0],
 39: [1.845, 11070.0],
 40: [1.74, 13920.0],
 41: [1.689, 25335.0],
 42: [6.633, 19899.0],
 43: [59.836, 837704.0

In [5]:
pars, areas, costs = gp.multidict(parcels)

In [6]:
budg = 1000000

In [7]:
#makes list of tuples for adjacency--can't be dictionary because not unique keys
adjacents = []
with open ('Adjacency.txt', mode = 'r') as csv_file:
    adjacents = [tuple(line) for line in csv.reader(csv_file)]
for idx, (x,y) in enumerate(adjacents): #converting to integers
    adjacents[idx] = (int(x), int(y))
#adjacents
#type(adjacents)

In [8]:
m = gp.Model('ParcelModel')

Academic license - for non-commercial use only - expires 2021-06-21
Using license file /Users/zrand/gurobi.lic


In [9]:
x = m.addVars(pars, obj = areas, vtype = GRB.BINARY, name = "parcel")

In [10]:
y = m.addVars(adjacents, vtype = GRB.BINARY, name = "flow")

In [11]:
B = m.addConstr(sum(costs[i]*x[i]for i in pars) <= budg, name = "budget")

In [12]:
core = m.addConstr(x[23] == 1, name = "core") #parcel 23 must be included in the reserve

In [13]:
for j in parcels:
    A = [item[0] for item in adjacents
            if item[1]==j]
    m.addConstr(sum(y[i,j] for i in A) <= len(A)*x[j], name = str("no_flow"+j)

In [14]:
for i in parcels:
    A = [item[1] for item in adjacents
            if item[0]==i]
    m.addConstr(sum(y[i,j] for j in A) <= x[i], name = str("one_flow" + i)

In [15]:
 #not sure why this one doesn't work
m.addConstr(sum(y[i,'*'] for i in parcels) == (sum(x[i])-1) for i in parcels, name = "connected")

SyntaxError: Generator expression must be parenthesized (<ipython-input-15-d30f8c634af9>, line 2)

In [16]:
#z = m.addVars(adjacents, name = "tail_length_contribution")
#w = m.addVars(parcels, name = "tail_length")

In [17]:
#this doesn't work yet
for i in parcels:
    A = [item[1] for item in adjacents
            if item[0]==i]
    for j in A:
        if j != i:
            m.addConstr(z[i,j] >= w[i] + 1 - m*(1 - y[i,j]), name = "tail") #what do we put for m?

TypeError: must be real number, not Model

In [15]:
m.setObjective(x.prod(areas), GRB.MAXIMIZE)

In [16]:
m.write('ParcelModel.lp')

In [ ]:
#note some things in the output file look SUPER weird so definitely need to fix some things.